In [1]:
%pip install tensorflow==2.10
%pip install scikit-learn
%pip install matplotlib
%matplotlib inline
%pip install keras_tuner

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.
^C
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


### Import Libraries

In [2]:
import os
import cv2
import glob
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.layers import Dense
from keras.models import Sequential
from keras.preprocessing import image
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from keras.layers import Convolution2D,Dense,MaxPool2D,Activation,Dropout,Flatten
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D

### GPU Acceleration Check

In [ ]:
print("TensorFlow version:", tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
physical_devices = tf.config.list_physical_devices()
print("Physical Devices:", physical_devices)

TensorFlow version: 2.10.0
Num GPUs Available:  0
Physical Devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


### Load and Split Prepared Dara

In [ ]:
# Load the data
x_input = np.load('prepared_data/x_input_combined.npy')
y_output = np.load('prepared_data/y_output_combined.npy')

# Ensure y is one-hot encoded
y_output = to_categorical(y_output, num_classes=4)

# Resizes images to 299 x 299 for optimal size for InceptionV3
x_input_resized = np.array([tf.image.resize(image, (299, 299)).numpy() for image in x_input])

Split dataset into training and testing with a 80/20 ratio

In [ ]:
# Split the data into training and testing sets
x_input_train, x_input_test, y_output_train, y_output_test = train_test_split(x_input_resized, y_output, test_size=0.2, random_state=42)

# Printing Dimensions of data
print(f'X Training Data\nNumber of Rows: {x_input_train.shape[0]}\nPixel Dimensions: {x_input_train.shape[1]}x{x_input_train.shape[2]}x{x_input_train.shape[3]}\n')
print(f'Y Training Data\nNumber of Rows: {y_output_train.shape[0]}\nNumber of Classes: {y_output_train.shape[1]}\n')
print(f'X Testing Data\nNumber of Rows: {x_input_test.shape[0]}\nPixel Dimensions: {x_input_test.shape[1]}x{x_input_test.shape[2]}x{x_input_test.shape[3]}\n')
print(f'Y Testing Data\nNumber of Rows: {y_output_test.shape[0]}\nNumber of Classes: {y_output_test.shape[1]}')

X Training Data
Number of Rows: 1536
Pixel Dimensions: 299x299x3

Y Training Data
Number of Rows: 1536
Number of Classes: 4

X Testing Data
Number of Rows: 384
Pixel Dimensions: 299x299x3

Y Testing Data
Number of Rows: 384
Number of Classes: 4


### Load InceptionV3 Model

In [ ]:
IMAGE_SIZE = [299, 299]

# Load the InceptionV3 model with weights pre-trained on ImageNet
inception = InceptionV3(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

# Freeze the existing weights
for layer in inception.layers:
    layer.trainable = False

# Add your own classification head to the model
x = GlobalAveragePooling2D()(inception.output)
x = Dense(256, activation='relu')(x)
output = Dense(4, activation='softmax')(x)

# Create a new model with the InceptionV3 base and your classification head
model = Model(inputs=inception.input, outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Display the summary of the model
model.summary()


NameError: name 'InceptionV3' is not defined

### Train Model

In [ ]:
history = model.fit(
    x_input_train, y_output_train,
    steps_per_epoch=None,
    epochs=30,
    validation_data=(x_input_test, y_output_test),
    validation_steps=None,
    verbose=1,
    callbacks=[ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=3, min_lr=0.001)],
    use_multiprocessing=False,
    shuffle=True
)


Epoch 1/30


48/48 [==============================] - 86s 2s/step - loss: 0.4226 - accuracy: 0.8509 - val_loss: 0.4597 - val_accuracy: 0.8333 - lr: 0.0010
Epoch 2/30
48/48 [==============================] - 97s 2s/step - loss: 0.3266 - accuracy: 0.8880 - val_loss: 0.4386 - val_accuracy: 0.8438 - lr: 0.0010
Epoch 3/30
48/48 [==============================] - 96s 2s/step - loss: 0.2376 - accuracy: 0.9219 - val_loss: 0.3522 - val_accuracy: 0.8672 - lr: 0.0010
Epoch 4/30
48/48 [==============================] - 90s 2s/step - loss: 0.1875 - accuracy: 0.9440 - val_loss: 0.4185 - val_accuracy: 0.8516 - lr: 0.0010
Epoch 5/30
48/48 [==============================] - 88s 2s/step - loss: 0.1626 - accuracy: 0.9492 - val_loss: 0.3077 - val_accuracy: 0.9089 - lr: 0.0010
Epoch 6/30
48/48 [==============================] - 88s 2s/step - loss: 0.1283 - accuracy: 0.9688 - val_loss: 0.3828 - val_accuracy: 0.8594 - lr: 0.0010
Epoch 7/30
48/48 [==============================] - 88s 2s/step - loss: 0.1005 - accuracy: 0.

## Evaluate Model

In [ ]:
# Evaluate the model on training data
result_train = model.evaluate(x_input_train, y_output_train)
print(dict(zip(model.metrics_names, result_train)))

# Evaluate the model on testing data
result_test = model.evaluate(x_input_test, y_output_test)
print(dict(zip(model.metrics_names, result_test)))


NameError: name 'model' is not defined

## Plot training and accuracy graphs

In [3]:
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy vs Epoch')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

plt.plot(history.history['loss'], label='Training loss')
plt.plot(history.history['val_loss'], label='Validation loss')
plt.title('Loss vs Epoch')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

NameError: name 'history' is not defined

## Confusion Matrix

In [ ]:
predictions = model.predict(x_input_test)
predicted_labels = np.argmax(predictions, axis=1)
true_labels = np.argmax(y_output_test, axis=1)
conf_matrix = confusion_matrix(true_labels, predicted_labels)

class_names = ['Class 1', 'Class 2', 'Class 3', 'Class 4']  # replace with your class names
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()


## Classification Report

In [ ]:
report = classification_report(true_labels, predicted_labels, target_names=class_names)
print("Classification Report:")
print(report)
